A neural network consist of cnn layer (Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [ ]:
import numpy as np
import codecs
import os
import random
import pandas
import pandas as pd
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Preprocessing

## Preprocessing

In [ ]:
! pip install hazm

In [ ]:
from hazm import *
normalizer=Normalizer()
stemmer=Stemmer()
lemmatizer=Lemmatizer()

def Normalizer(text):

    # normalizer = Normalizer()
    normalized = normalizer.normalize(text)

    return normalized


def SentenceSplitter_Tokenizer(text):

    Tokens=list(word_tokenize(text))

    return Tokens

def Stemmer(token):

    stemming=[]
    for word in token:
        stemming.append(lemmatizer.lemmatize(word))

    return stemming

def RemoveStopwords_Punc(token):

    with open('../Data/stopwords') as f:
        sw = [line.rstrip().replace(" ","").replace("\u200c","") for line in f]
    # print(sw)

    punctuation = '!"#$%&\'()*+–,-./:;<=>?@[\\]^_`{|}~،؟«؛'
    result=[]
    for word in token:
        for s in punctuation:
            if s in word:
                word=word.replace(s,"")
            # print(word)
        if word.replace(" ","").replace("\u200c","") in sw:
            continue

        result.append(word)

    # print(result)
    return result

def EmojiRemoving(text):

    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    text= re.sub(emoj, '', text)

    return text


In [ ]:
data = pandas.read_excel('../labels.xlsx')
data = data.rename(columns={'labels': 'tag','text':'body'})
data = data.dropna().reset_index(drop=True)
data.head()

,Unnamed: 0,body
0,0,به عنوان یک برنامه نویس اندروید چطوری می توانی...
1,1,مهدی سادات حسینی دانشجو دوست کار دوست کامپیوتر...
2,2,کد مخف اندروید کد مخف اندروید اندروید کد کد پر...
3,3,با سلام خدمت شما خوانندگان عزیز تو این مطلب که...
4,4,حالا این که وقتی دیجیتال مارکتر به مانیتور خیر...


In [ ]:
# # counter
# count_data = data.copy()
# count_data = count_data.dropna().reset_index(drop=True)
# count = {}
# tags_list = pd.read_excel('../Tags.xlsx',header = None)
# tags_list.columns = ['tag']
# for _,tag in tags_list.iterrows():
#   if tag['tag']!='ویرگول' and tag['tag']!=None and tag['tag']!='':
#     count[re.sub(' ','',tag['tag'].lower())] = 0

# for indx,d in count_data.iterrows():
#   tags = re.sub('\'|\[|\]','',d['tag']).split(',')
  
#   for tag in tags:
#     if tag.strip().lower() in tags_list.values:
#       count[tag.strip().lower().replace(' ','')]+=1 
# count

In [ ]:
downly_tags = [i for i in list(count.keys()) if count[i]<100]
less_than_80 = [i for i in list(count.keys()) if count[i]<80]
less_than_80

['برنامهنویسی',
 'پایتون',
 'اندروید',
 'جاوااسکریپت',
 'هوشمصنوعی',
 'ریاکت',
 'لینوکس',
 'php',
 'یادگیریماشین',
 'لاراول',
 'نرمافزار',
 'آموزشبرنامهنویسی',
 'وب',
 'کامپیوتر',
 'مهندسینرمافزار',
 'استارتاپ',
 'تکنولوژی',
 'برنامهنویسیاندروید',
 'یادگیری',
 'امنیت',
 'گیت',
 'جاوا',
 'فلاتر',
 'کاتلین',
 'طراحیوب',
 'کسبوکار',
 'توسعهدهنده',
 'کدنویسی',
 'داکر',
 'nodejs',
 'علمداده',
 'فرانتاند',
 'زبانبرنامهنویسی',
 'golang',
 'سئو',
 'استخدام',
 'کد',
 'آموزشپایتون',
 'الگوریتم',
 'اینترنت',
 'css',
 'گوگل',
 'طراحیسایت',
 'api',
 'مصاحبه',
 'redux',
 'دادهکاوی',
 'موفقیت',
 'یادگیریعمیق',
 'هک',
 'وردپرس',
 'تجربهکاربری',
 'رابطکاربری',
 'دیجیتالمارکتینگ',
 'تحلیلداده',
 'ویندوز',
 'oop',
 'reactnative',
 'تلگرام',
 'گیتهاب',
 'ubuntu',
 'designpattern',
 'برنامهنویسیوب',
 'html',
 'سیشارپ',
 'شیگرایی',
 'کتاب',
 'جنگو',
 'اندرویداستودیو',
 'اینستاگرام',
 'فناوری',
 'مهارت',
 'devops',
 'ios',
 'توسعهوب',
 'اپلیکیشن',
 'c',
 'vscode']

In [ ]:
slice_size = 150
for i in range(len(data)):

    text=data.loc[i,"body"]

    # tags = data.loc[i,'tag']
    # tags = re.sub('\[|\]|\'|=|ویرگول',' ',tags)
    # tags_list = [t.strip() for t  in tags.split(',') if (t!=' ' and t!='' and (re.sub('[ ]*','',t) in list(count.keys())))]
    # if len(tags_list)==0:
    #   data.loc[i,'body'] = None    
    #   if i%3000==0:
    #     print("Record: up to "+str(i)+" from " ,str(len(data)),"is processed then breaked.")
    #   continue
    
    # remove half-space

    text = text.replace("\u200c", " ")
    text = text.replace("های ", " ها ی ")
    text = text.replace("ها ", " ها ")
    text=EmojiRemoving(text)
    normalized = Normalizer(text)
    split = SentenceSplitter_Tokenizer(normalized)
    tokens = RemoveStopwords_Punc(split)
    tokens = [t for t in tokens if len(t)>2]
    tokens_slice = []

    # balancing
    
    # min_count = min([count[tag.replace(' ','')] for tag in tags_list if (tag.replace(' ','') in list(count.keys()))])

    # if min_count<80 :
    #   loop_0 = len(tokens)//300
    #   loop_1 = int(min_count/(loop_0+1))

    #   for j in range(loop_0):
    #       d_temp = data.iloc[i,:].copy()

    #       for tag in tags_list:
    #         count[tag.replace(' ','')] += 1

    #       d_temp['body'] = tokens[j*300:min(len(tokens),(j+1)*300)]
    #       for _ in range(loop_1):
    #         data = data.append(d_temp)

    if len(tokens)> 2*slice_size:
      tokens_slice += tokens[:slice_size]
      tokens_slice += tokens[-slice_size:]
    else:
      tokens_slice += tokens

    if tokens_slice!= None:
      data.loc[i, "body"] = re.sub('[ ]*|\\u200c|','',str(tokens_slice)).replace('\\u200c','')
    else:
      data.loc[i, "body"] = None
      
    if i%3000==0:
      print("Record: up to "+str(i)+" from " ,str(len(data)),"is processed.")


Record: up to 0 from  3653 is processed.
Record: up to 3000 from  3653 is processed.


In [ ]:
# data.loc[169,'tag']

In [ ]:
data = data.dropna().reset_index(drop=True)
data.head()

,Unnamed: 0,body
0,0,"['برنامه','نویس','اندروید','چطوری','میتوانید',..."
1,1,"['مهدی','سادات','حسینی','دانشجو','دوست','کار',..."
2,2,"['مخف','اندروید','مخف','اندروید','اندروید','پر..."
3,3,"['سلام','خدمت','خوانندگان','عزیز','مطلب','نوشت..."
4,4,"['دیجیتال','مارکتر','مانیتور','کار','میu','۲۰۰..."


In [ ]:
# # counter
# count_data = data.copy()
# count_data = count_data.dropna().reset_index(drop=True)
# count = {}
# tags_list = pd.read_excel('../Tags.xlsx',header = None)
# tags_list.columns = ['tag']
# for _,tag in tags_list.iterrows():
#   if tag['tag']!='ویرگول' and tag['tag']!=None and tag['tag']!='':
#     count[re.sub(' ','',tag['tag'].lower())] = 0

# for indx,d in count_data.iterrows():
#   tags = re.sub('\'|\[|\]','',d['tag']).split(',')
  
#   for tag in tags:
#     if tag.strip().lower() in tags_list.values:
#       count[tag.strip().lower().replace(' ','')]+=1 
# # count

In [ ]:
# print(data.shape)
# data.loc[:100,['body','tag']].head() 

In [ ]:
data.to_csv('../pr_data.csv', index=False)

## FreqDist

In [ ]:
from nltk import FreqDist

In [ ]:
all_tokens = []
for body in data['body']:
  all_tokens+= re.sub('\'|\"|\[|\]| ',',',body).split(',')

In [ ]:
fd_all = FreqDist(all_tokens)
fd_all[0]

0

In [ ]:
keys = pd.DataFrame([(item,fd_all[item]) for item in fd_all])
keys.columns = ['word','freq']
keys = keys.sort_values(by=['freq'],ascending=False)
keys = keys.reset_index(drop=True)
keys.to_csv('../FreqDist_sorted.csv', index=False)

In [ ]:
keys.head()

,word,freq
0,,1450010
1,۲۰۰,38763
2,میu,12175
3,کار,5875
4,برنامه,5428


# Deep learning models

In [ ]:
x = data["body"].values.tolist()
# y = data.copy()
# labels = []
# tag=[]
# for item in y['tag']:
#   labels += [i for i in re.sub('\"|\[|\]|\'| |ویرگول|=','',item.lower()).split(",") if i!='' and i!=' ']
#   tag.append([i for i in re.sub('\"|\[|\]|\'| |ویرگول|=','',item.lower()).split(",") if i!='' and i!=' '])
# labels = list(set(labels))
# mlb = MultiLabelBinarizer()
# Y=mlb.fit_transform(tag)

In [ ]:
# len(labels)

In [ ]:
# len(labels)

In [ ]:
x[0]

"['برنامه','نویس','اندروید','چطوری','میتوانید','کارهایی','بردن','امنیت','اندروید','اپلیکیشن','بلد','کمک','رقبا','قدم','جلوتر','اطلااتی','مقاله','امنیت','اندروید','بتوانید','امنیت','سیستم','عامل','اندروید','بهبود','ببخشید','اشنا','مقاله','الکامکو','سایت','همراه','سیستم','عامل','اندروید','ویژگیها','امنیتی','داخلی','نتیجه','برنامهها','ساده','اندرویدی','عملیات','بهxa۰سیستم','فایل','عملیات','شبکه','نمیدهند','فرض','امن','گرفتهمیشوند','توسعه','برنامه','پیچیدهتر','مسیولیت','امنیت','برعهده','حریم','خصوصی','کاربران','محافظت','موضوعاتی','مقاله','اشاره','توجه','مقاله','امنیت','اندروید','قصد','روشهایی','ایمن','اپلیکیشن','اندروید','موقع','ساخت','کمک','معرفی','برنامهها','مخربی','سیستم','نصب','اسیب','ببینند']"

In [ ]:
import ast
X=[]
for i in x:
  X.append(ast.literal_eval(i))
print(X[0])

['برنامه', 'نویس', 'اندروید', 'چطوری', 'میتوانید', 'کارهایی', 'بردن', 'امنیت', 'اندروید', 'اپلیکیشن', 'بلد', 'کمک', 'رقبا', 'قدم', 'جلوتر', 'اطلااتی', 'مقاله', 'امنیت', 'اندروید', 'بتوانید', 'امنیت', 'سیستم', 'عامل', 'اندروید', 'بهبود', 'ببخشید', 'اشنا', 'مقاله', 'الکامکو', 'سایت', 'همراه', 'سیستم', 'عامل', 'اندروید', 'ویژگیها', 'امنیتی', 'داخلی', 'نتیجه', 'برنامهها', 'ساده', 'اندرویدی', 'عملیات', 'بهxa۰سیستم', 'فایل', 'عملیات', 'شبکه', 'نمیدهند', 'فرض', 'امن', 'گرفتهمیشوند', 'توسعه', 'برنامه', 'پیچیدهتر', 'مسیولیت', 'امنیت', 'برعهده', 'حریم', 'خصوصی', 'کاربران', 'محافظت', 'موضوعاتی', 'مقاله', 'اشاره', 'توجه', 'مقاله', 'امنیت', 'اندروید', 'قصد', 'روشهایی', 'ایمن', 'اپلیکیشن', 'اندروید', 'موقع', 'ساخت', 'کمک', 'معرفی', 'برنامهها', 'مخربی', 'سیستم', 'نصب', 'اسیب', 'ببینند']


In [ ]:
len(labels)

78

In [ ]:
sentence_maxlen =386
print('sentence maxlen', 386)

sentence maxlen 386


In [ ]:
freq_dist = keys
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

توسعه


In [ ]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [ ]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 50580
28421


In [ ]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
# vecY=Y

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
# print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# # print(vecX[0])

# Model is generated before

In [ ]:
model = load_model('bestModelForTest_cnn_4fc.h5')
# model.save('CNN_1_no_binary.h5')

In [ ]:
pred=model.predict(vecX)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
pred_df = pd.DataFrame(pred)
pred_df.to_csv('Deep_Learning_result.cav',incex=False

((3653, 78), (3653, 78))

In [ ]:
print(pred[0])#example

[0.03054118 0.10744637 0.05683747 0.02625084 0.00784066 0.02593207
 0.03781939 0.0169543  0.01433051 0.07726216 0.04778329 0.00067624
 0.00426912 0.02163193 0.01537514 0.02275059 0.00952297 0.022699
 0.01140109 0.01233888 0.04593882 0.07424015 0.02500221 0.06028041
 0.01875052 0.03680861 0.02225903 0.01714835 0.01541653 0.01372537
 0.01028144 0.08021867 0.01701614 0.01447877 0.03487262 0.05486584
 0.04597512 0.00664613 0.01562279 0.01390362 0.01133084 0.02807444
 0.02618107 0.01156065 0.01497591 0.03156281 0.01551053 0.02716434
 0.02448991 0.01374757 0.01114982 0.02247164 0.01833495 0.04365975
 0.02941796 0.01313087 0.01626781 0.01040107 0.06137931 0.03325981
 0.01492712 0.04119733 0.03923252 0.01723284 0.02961957 0.02081352
 0.01567277 0.03975511 0.21347573 0.04657888 0.02123901 0.05398211
 0.02421138 0.01328287 0.01251704 0.03230238 0.01135305 0.01386419]


In [ ]:
pred_df =  pd.read_csv('Deep_Learning_result.csv')
pred = pred_df.values
y_pred=[]
measure = np.mean(pred[0]) + .01*np.sqrt(np.var(pred[0]))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

y_pred_df =  pd.DataFrame(y_pred)
y_pred_df.to_csv('Deep_Learning_result.csv')

In [15]:
y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0
1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0
2,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0
3,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0
4,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0


In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(vecY, y_pred))
print(classification_report(vecY, y_pred))

accuracy= 0.0
              precision    recall  f1-score   support

           0       0.02      0.77      0.03        65
           1       0.01      1.00      0.03        49
           2       0.02      1.00      0.03        61
           3       0.00      0.00      0.00        44
           4       0.20      0.02      0.03        59
           5       0.04      0.18      0.06        66
           6       0.01      0.94      0.03        53
           7       0.00      0.00      0.00        62
           8       0.00      0.00      0.00        41
           9       0.01      1.00      0.02        45
          10       0.02      0.23      0.04       104
          11       0.00      0.00      0.00        60
          12       0.00      0.00      0.00        45
          13       0.01      0.02      0.01        60
          14       0.00      0.00      0.00        44
          15       0.01      0.04      0.02        93
          16       0.00      0.00      0.00        71
          17 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
measure

0.03114856341853738

In [ ]:
print(len(vecY),len(y_pred))
print(len(vecY[0]),len(y_pred[0]))

3608 3608
78 78
